In [2]:
import numpy as np
import pandas as pd
import os
from pathlib import Path
from tqdm import tqdm

In [4]:
from pathlib import Path

# Get the current working directory
current_path = Path.cwd()

# Get the parent directory
parent_path = current_path
data_paths = parent_path / 'datas'  

print(data_paths)
#
train_path = data_paths / 'train.csv'
test_path = data_paths / 'test.csv'

train_path_ts = data_paths / 'series_train.parquet'
test_path_ts = data_paths / 'series_test.parquet'

/Users/nayelbenabdesadok/GitProjects/CMI_problematic_internet_use/datas


In [5]:
# Load datasets

train = pd.read_csv(train_path)
test = pd.read_csv(test_path)

In [62]:
features_to_keep = ['id', 'Basic_Demos-Age', 'Basic_Demos-Sex','CGAS-CGAS_Score', 'Physical-BMI', 'Physical-Diastolic_BP','Physical-HeartRate','Physical-Systolic_BP','Fitness_Endurance-Max_Stage','Fitness_Endurance-Time_Sec', 
                    'FGC-FGC_CU_Zone','FGC-FGC_GSND_Zone','FGC-FGC_GSD_Zone', 'FGC-FGC_PU_Zone', 'FGC-FGC_SRL_Zone', 'FGC-FGC_SRR_Zone','FGC-FGC_TL_Zone',
                    'BIA-BIA_Activity_Level_num', 'BIA-BIA_BMC', 'BIA-BIA_BMI', 'BIA-BIA_BMR', 'BIA-BIA_DEE', 'BIA-BIA_ECW', 'BIA-BIA_FFM',
                    'BIA-BIA_FFMI', 'BIA-BIA_FMI', 'BIA-BIA_Fat', 'BIA-BIA_Frame_num','BIA-BIA_ICW', 'BIA-BIA_LDM', 'BIA-BIA_LST', 'BIA-BIA_SMM', 'BIA-BIA_TBW',
                    'PCIAT-PCIAT_03', 'PCIAT-PCIAT_04', 'PCIAT-PCIAT_05', 'PCIAT-PCIAT_06', 'PCIAT-PCIAT_07', 'PCIAT-PCIAT_08', 'PCIAT-PCIAT_09', 'PCIAT-PCIAT_10',
                    'PCIAT-PCIAT_11', 'PCIAT-PCIAT_12', 'PCIAT-PCIAT_13', 'PCIAT-PCIAT_14','PCIAT-PCIAT_15', 'PCIAT-PCIAT_16', 'PCIAT-PCIAT_17', 'PCIAT-PCIAT_18','PCIAT-PCIAT_19', 'PCIAT-PCIAT_20',
                    'SDS-SDS_Total_Raw', 'SDS-SDS_Total_T', 'PreInt_EduHx-computerinternet_hoursday',
                    'sii']


# Select the specified columns from the train DataFrame
train_selected = train[features_to_keep].copy()


In [63]:
columns_to_drop = []
train_selected=train_selected.dropna(subset=["sii"])

for column in features_to_keep:
    # Calculate the ratio of NaN values
    nan_count = train[column].isna().sum()
    total_count = len(train[column])
    ratio = nan_count / total_count
    print(f"The ratio for {column} is {ratio}")
    
    if ratio > 0.7:
        columns_to_drop.append(column)

# Drop the columns with NaN ratio greater than 0.7
train_selected.drop(columns=columns_to_drop, inplace=True)

print(f"Columns dropped: {columns_to_drop}")

The ratio for id is 0.0
The ratio for Basic_Demos-Age is 0.0
The ratio for Basic_Demos-Sex is 0.0
The ratio for CGAS-CGAS_Score is 0.3886363636363636
The ratio for Physical-BMI is 0.23686868686868687
The ratio for Physical-Diastolic_BP is 0.25404040404040407
The ratio for Physical-HeartRate is 0.25075757575757573
The ratio for Physical-Systolic_BP is 0.25404040404040407
The ratio for Fitness_Endurance-Max_Stage is 0.8123737373737374
The ratio for Fitness_Endurance-Time_Sec is 0.8131313131313131
The ratio for FGC-FGC_CU_Zone is 0.42373737373737375
The ratio for FGC-FGC_GSND_Zone is 0.7318181818181818
The ratio for FGC-FGC_GSD_Zone is 0.7315656565656565
The ratio for FGC-FGC_PU_Zone is 0.4265151515151515
The ratio for FGC-FGC_SRL_Zone is 0.42752525252525253
The ratio for FGC-FGC_SRR_Zone is 0.427020202020202
The ratio for FGC-FGC_TL_Zone is 0.42297979797979796
The ratio for BIA-BIA_Activity_Level_num is 0.49722222222222223
The ratio for BIA-BIA_BMC is 0.49722222222222223
The ratio for BI

In [64]:
# Identify float type columns
float_columns = train_selected.select_dtypes(include=['float64']).columns

print(float_columns)

# Group by 'Basic_Demos-Age' and 'Basic_Demos-Sex'
grouped = train_selected.groupby(['Basic_Demos-Age', 'Basic_Demos-Sex'])

# Function to fill NaN values with the median of the group
def fill_na_with_median(group):
    for column in float_columns:
        median_value = group[column].median()
        group[column].fillna(median_value, inplace=True)
    return group

# Apply the function to each group
train_selected = grouped.apply(fill_na_with_median)


print(train_selected)

Index(['CGAS-CGAS_Score', 'Physical-BMI', 'Physical-Diastolic_BP',
       'Physical-HeartRate', 'Physical-Systolic_BP', 'FGC-FGC_CU_Zone',
       'FGC-FGC_PU_Zone', 'FGC-FGC_SRL_Zone', 'FGC-FGC_SRR_Zone',
       'FGC-FGC_TL_Zone', 'BIA-BIA_Activity_Level_num', 'BIA-BIA_BMC',
       'BIA-BIA_BMI', 'BIA-BIA_BMR', 'BIA-BIA_DEE', 'BIA-BIA_ECW',
       'BIA-BIA_FFM', 'BIA-BIA_FFMI', 'BIA-BIA_FMI', 'BIA-BIA_Fat',
       'BIA-BIA_Frame_num', 'BIA-BIA_ICW', 'BIA-BIA_LDM', 'BIA-BIA_LST',
       'BIA-BIA_SMM', 'BIA-BIA_TBW', 'PCIAT-PCIAT_03', 'PCIAT-PCIAT_04',
       'PCIAT-PCIAT_05', 'PCIAT-PCIAT_06', 'PCIAT-PCIAT_07', 'PCIAT-PCIAT_08',
       'PCIAT-PCIAT_09', 'PCIAT-PCIAT_10', 'PCIAT-PCIAT_11', 'PCIAT-PCIAT_12',
       'PCIAT-PCIAT_13', 'PCIAT-PCIAT_14', 'PCIAT-PCIAT_15', 'PCIAT-PCIAT_16',
       'PCIAT-PCIAT_17', 'PCIAT-PCIAT_18', 'PCIAT-PCIAT_19', 'PCIAT-PCIAT_20',
       'SDS-SDS_Total_Raw', 'SDS-SDS_Total_T',
       'PreInt_EduHx-computerinternet_hoursday', 'sii'],
      dtype='object')


/var/folders/sg/lshv0q7145ggm8z9kc3b1y4r0000gn/T/ipykernel_66358/3596357927.py:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  group[column].fillna(median_value, inplace=True)


                                            id  Basic_Demos-Age  \
Basic_Demos-Age Basic_Demos-Sex                                   
5               0               0     00008ff9                5   
                                210   0bd89878                5   
                                267   0f6b71c4                5   
                                297   1184a14a                5   
                                349   1455e657                5   
...                                        ...              ...   
21              1               2012  7fe0a99a               21   
                                2337  93c72a97               21   
                                3134  c9ad9c75               21   
22              1               1298  50a90725               22   
                                3297  d50a8c3c               22   

                                      Basic_Demos-Sex  CGAS-CGAS_Score  \
Basic_Demos-Age Basic_Demos-Sex                       

/var/folders/sg/lshv0q7145ggm8z9kc3b1y4r0000gn/T/ipykernel_66358/3596357927.py:17: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  train_selected = grouped.apply(fill_na_with_median)


In [94]:
from sklearn.impute import KNNImputer

# Drop ID columns or any non-relevant columns

ids = train_selected['id']
train_selected_without_ids = train_selected.drop(['id'], axis=1)

# Initialize the KNNImputer
imputer = KNNImputer(n_neighbors=1)

# Fit and transform the data
imputed_data = imputer.fit_transform(train_selected_without_ids)

# Optionally, convert the result back to a DataFrame
imputed_df = pd.DataFrame(imputed_data, columns=train_selected_without_ids.columns)

imputed_df['id'] = ids.values

In [95]:
features_to_keep = imputed_df.columns

for column in features_to_keep:
    # Calculate the ratio of NaN values
    nan_count = imputed_df[column].isna().sum()
    total_count = len(train[column])
    ratio = nan_count / total_count
    print(f"The ratio for {column} is {ratio}")

The ratio for Basic_Demos-Age is 0.0
The ratio for Basic_Demos-Sex is 0.0
The ratio for CGAS-CGAS_Score is 0.0
The ratio for Physical-BMI is 0.0
The ratio for Physical-Diastolic_BP is 0.0
The ratio for Physical-HeartRate is 0.0
The ratio for Physical-Systolic_BP is 0.0
The ratio for FGC-FGC_CU_Zone is 0.0
The ratio for FGC-FGC_PU_Zone is 0.0
The ratio for FGC-FGC_SRL_Zone is 0.0
The ratio for FGC-FGC_SRR_Zone is 0.0
The ratio for FGC-FGC_TL_Zone is 0.0
The ratio for BIA-BIA_Activity_Level_num is 0.0
The ratio for BIA-BIA_BMC is 0.0
The ratio for BIA-BIA_BMI is 0.0
The ratio for BIA-BIA_BMR is 0.0
The ratio for BIA-BIA_DEE is 0.0
The ratio for BIA-BIA_ECW is 0.0
The ratio for BIA-BIA_FFM is 0.0
The ratio for BIA-BIA_FFMI is 0.0
The ratio for BIA-BIA_FMI is 0.0
The ratio for BIA-BIA_Fat is 0.0
The ratio for BIA-BIA_Frame_num is 0.0
The ratio for BIA-BIA_ICW is 0.0
The ratio for BIA-BIA_LDM is 0.0
The ratio for BIA-BIA_LST is 0.0
The ratio for BIA-BIA_SMM is 0.0
The ratio for BIA-BIA_TBW 

In [96]:
imputed_df=imputed_df.drop("id",axis=1)

In [97]:
imputed_df

,Basic_Demos-Age,Basic_Demos-Sex,CGAS-CGAS_Score,Physical-BMI,Physical-Diastolic_BP,Physical-HeartRate,Physical-Systolic_BP,FGC-FGC_CU_Zone,FGC-FGC_PU_Zone,FGC-FGC_SRL_Zone,...,PCIAT-PCIAT_15,PCIAT-PCIAT_16,PCIAT-PCIAT_17,PCIAT-PCIAT_18,PCIAT-PCIAT_19,PCIAT-PCIAT_20,SDS-SDS_Total_Raw,SDS-SDS_Total_T,PreInt_EduHx-computerinternet_hoursday,sii
0,5.0,0.0,51.0,16.877316,69.0,88.5,115.0,0.0,0.0,0.0,...,4.0,4.0,4.0,4.0,2.0,4.0,40.0,56.0,3.0,2.0
1,5.0,0.0,60.0,15.968632,66.0,83.0,107.0,0.0,0.0,0.0,...,0.0,4.0,1.0,3.0,0.0,0.0,40.0,56.0,2.0,0.0
2,5.0,0.0,50.0,12.926988,93.0,86.0,157.0,0.0,0.0,1.0,...,0.0,2.0,1.0,1.0,0.0,0.0,40.0,56.0,2.0,0.0
3,5.0,0.0,51.0,16.113052,66.0,82.0,92.0,0.0,0.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,40.0,56.0,2.0,0.0
4,5.0,0.0,60.0,24.023506,78.0,116.0,131.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,40.0,56.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2731,21.0,1.0,61.0,28.255461,66.0,76.0,122.0,0.0,0.0,1.0,...,2.0,0.0,0.0,0.0,0.0,0.0,30.0,43.0,3.0,0.0
2732,21.0,1.0,50.0,25.557586,54.0,83.0,129.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,32.0,46.0,3.0,0.0
2733,21.0,1.0,55.0,30.953337,74.0,66.0,115.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,32.0,46.0,3.0,0.0
2734,22.0,1.0,63.0,34.509396,95.0,120.0,144.0,0.0,0.0,1.0,...,4.0,4.0,1.0,1.0,1.0,3.0,50.0,69.0,2.0,1.0


In [98]:
import xgboost as xgb
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [99]:
X=imputed_df.drop("sii",axis=1)
y=imputed_df["sii"]

In [100]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [101]:
model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss')
model.fit(X_train, y_train)

/Users/nayelbenabdesadok/opt/anaconda3/envs/kaggle_cmi/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [16:36:43] WARNING: /var/folders/c_/qfmhj66j0tn016nkx_th4hxm0000gp/T/abs_b6qk1lz_ug/croot/xgboost-split_1724073748391/work/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, objective='multi:softprob', ...)

In [102]:
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Précision du modèle : {accuracy * 100:.2f}%')

Précision du modèle : 90.33%


In [84]:
test_to_keep=test[features_to_keep]

KeyError: "['PCIAT-PCIAT_03', 'PCIAT-PCIAT_04', 'PCIAT-PCIAT_05', 'PCIAT-PCIAT_06', 'PCIAT-PCIAT_07', 'PCIAT-PCIAT_08', 'PCIAT-PCIAT_09', 'PCIAT-PCIAT_10', 'PCIAT-PCIAT_11', 'PCIAT-PCIAT_12', 'PCIAT-PCIAT_13', 'PCIAT-PCIAT_14', 'PCIAT-PCIAT_15', 'PCIAT-PCIAT_16', 'PCIAT-PCIAT_17', 'PCIAT-PCIAT_18', 'PCIAT-PCIAT_19', 'PCIAT-PCIAT_20', 'sii'] not in index"

# Time Series

In [6]:
ts_train=pd.read_parquet(train_path_ts)

: 